In [1]:
#from cityscapes_dataset import CityscapesDataset
from nyuv2_dataset import NYUv2Dataset
from cityscapes_dataset import CityscapesDataset
import torch
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from cross_stitchnet import CrossStitchNet
from densenet import DenseNet
from mtan import MTAN
from splitnet import SplitNet
from trainer import Trainer
from utils import count_params

2025-03-16 00:35:11.181902: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-16 00:35:11.242546: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-03-16 00:35:12.166850: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
LABELS = 7
BATCH_SIZE = 8
LR = 1e-4

In [3]:
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda:1


In [4]:
# print("NYUv2 Dataset")
# nyuv2_train = NYUv2Dataset(root="../dataset/nyuv2_preprocessed", split='train')
# classes = nyuv2_train.get_classes()

# nyuv2_val = NYUv2Dataset(root="../dataset/nyuv2_preprocessed", split='val')
# train_dl = DataLoader(nyuv2_train, batch_size=BATCH_SIZE, shuffle=True)
# val_dl = DataLoader(nyuv2_val, batch_size=BATCH_SIZE, shuffle=False)

# for image, out in train_dl:
#     print('Image: ' + str(list(image.shape)) + ',' + ' Label: ' + str(list(out['segmentation'].shape)) + ',' + ' Depth: ' + str(list(out['depth'].shape)) + ',' + ' Normals: ' + str(list(out['normal'].shape)))
#     print(f'Image: {image.max().item()}, {image.min().item()}') 
#     print('Label: ' +  str(out['segmentation'].max().item()) + ',' + str(out['segmentation'].min().item()))
#     print('Depth: ' + str(out['depth'].max().item()) + ', ' + str(out['depth'].min().item()))
#     print('Normals: ' + str(out['normal'].max().item()) + ', ' + str(out['normal'].min().item()))
#     break
# ax, fig = plt.subplots(4, figsize=(10, 10))
# fig[0].imshow(image[0].permute(1, 2, 0))
# fig[1].imshow(label[0])
# fig[2].imshow(depth[0])
# fig[3].imshow(normals[0].permute(1, 2, 0))
# plt.show()

In [5]:
print("Cityscapes Dataset")
cityscapes_train = CityscapesDataset(root="../dataset/cityscapes_preprocessed")
cityscapes_val = CityscapesDataset(root="../dataset/cityscapes_preprocessed", split='val')
train_dl = DataLoader(cityscapes_train, batch_size=BATCH_SIZE, shuffle=True)
val_dl = DataLoader(cityscapes_val, batch_size=BATCH_SIZE, shuffle=False)
classes = cityscapes_train.get_classes()
for image, out in train_dl:
    print('Image: ' + str(list(image.shape)) + ',' + ' Label: ' + str(list(out['segmentation'].shape)) + ',' + ' Depth: ' + str(list(out['depth'].shape)))
    print(f'Image: {image.max().item()}, {image.min().item()}') 
    print('Label: ' +  str(out['segmentation'].max().item()) + ', ' + str(out['segmentation'].min().item()))
    print('Depth: ' + str(out['depth'].max().item()) + ', ' + str(out['depth'].min().item()))
    break

Cityscapes Dataset
Image: [8, 3, 128, 256], Label: [8, 128, 256], Depth: [8, 128, 256]
Image: 1.0, 0.0
Label: 6.0, -1.0
Depth: 0.4922102689743042, 0.0


In [6]:
# cityscapes_train = CityscapesDataset(root="../cityscapes_preprocessed", split="train",labels=LABELS)
# train_dl = DataLoader(cityscapes_train, batch_size=BATCH_SIZE, shuffle=False)
# for image, label, depth in train_dl:
#     print(f'Image: {image.shape}, Label: {label.shape}, Depth: {depth.shape}')
#     print(f'Image: {image.max()}, {image.min()}') 
#     print(f'Label: {label.max()}, {label.min()}')
#     print(f'Depth: {depth.max()}, {depth.min()}')
#     break
# ax, fig = plt.subplots(3, figsize=(10, 10))
# fig[0].imshow(image[0].permute(1, 2, 0))
# fig[1].imshow(label[0])
# fig[2].imshow(depth[0])
# plt.show()

# cityscapes_val = CityscapesDataset(root="../cityscapes_preprocessed", split="val", labels=LABELS)
# val_dl = DataLoader(cityscapes_val, batch_size=BATCH_SIZE, shuffle=False)

In [7]:
model = CrossStitchNet(classes=classes, tasks=['segmentation', 'depth'], depth_activation='sigmoid').to(device)
print(f"{model.name} has {count_params(model)} parameters")
# opt = torch.optim.Adam(model.parameters(), lr=1e-4)
# trainer = Trainer(model, opt)
# trainer.train(train_dl, val_dl, epochs=2, save=True)

cross_stitch_seg_dep has 50094350 parameters


In [8]:
opt = torch.optim.Adam(model.parameters(), lr=LR)
trainer = Trainer(model, opt, "cityscapes", device)

In [9]:
trainer.train(train_dl, val_dl, epochs=10, save=True, check=1, grad=True, dwa=True)

100%|██████████| 372/372 [02:38<00:00,  2.35it/s]


Epoch 0/9 - Train Total Loss: 0.0012
lambda_segmentation : 1.0 - Train Loss segmentation: 0.7828
lambda_depth : 1.0 - Train Loss depth: 0.0713
miou: 0.21356379985809326
pix_acc: 0.7854910492897034
mae: 0.07125096023082733
mre: 435.770751953125
Gradient Norm: 1.1089879363601105



100%|██████████| 63/63 [00:09<00:00,  6.72it/s]


Val Loss segmentation: 0.7485
Val Loss depth: 0.0383
miou: 0.27377837896347046
pix_acc: 0.8693028092384338
mae: 0.032924480736255646
mre: 108.64857482910156




100%|██████████| 372/372 [02:40<00:00,  2.32it/s]


...

Epoch 6/9 - Train Total Loss: 0.0008
lambda_segmentation : 0.9918993711471558 - Train Loss segmentation: 0.2391
lambda_depth : 1.0081006288528442 - Train Loss depth: 0.0190
miou: 0.3900740444660187
pix_acc: 0.9273210763931274
mae: 0.018989436328411102
mre: 44.5858039855957
Gradient Norm: 0.856705412989393



100%|██████████| 63/63 [00:09<00:00,  6.75it/s]


Val Loss segmentation: 0.4419
Val Loss depth: 0.0271
miou: 0.3492293655872345
pix_acc: 0.9089809060096741
mae: 0.02010798454284668
mre: 56.260623931884766




 99%|█████████▉| 370/372 [02:39<00:00,  2.32it/s]


KeyboardInterrupt: 

In [ ]:
# for x, y in train_dl:
#     x = x.to(device).to(torch.float)
#     y = {k: v.to(device) for k, v in y.items()}
#     y['segmentation'] = y['segmentation'].to(torch.long)
#     y['depth'] = y['depth'].to(torch.float)
#     out = model(x)
#     print(f"Segmentation: {out['segmentation'].shape}")
#     print(f"Depth: {out['depth'].shape}")
#     print(f"Normals: {out['normal'].shape}")
#     break

In [ ]:
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# tests = 10
# path = 'models/mtan/mtan_train10.pth'
# model = MTAN(classes=LABELS)
# model.load_state_dict(torch.load(path, weights_only=True))
# model.to(device)
# for i, (img, img_seg, img_depth) in enumerate(val_dl):
#     visualize_results_multitask(model, img[0], img_seg[0], img_depth[0], device)
#     if i == tests:
#         break